In [ ]:
# nbstripout is a tool to remove the output from Jupyter notebooks
#!nbstripout --install
!export PYTHONWARNINGS="ignore:NotOpenSSLWarning"

from dotenv import load_dotenv
load_dotenv()
from langchain.document_loaders import PyPDFLoader
from openai import OpenAI

In [ ]:
def call_openai_api(path):
    k = 1
    global response_string, response_res, json_array_one
    json_array_one = []
    loader = PyPDFLoader(path)
    pages = loader.load_and_split()
    system_message = (
        "In this scenario, you are a chemist with a focus on natural products, tasked with analyzing a scientific paper to identify a specific molecule. Your response must be structured in JSON format, capturing key information about the molecules in question. This includes: `IUPAC_nomenclature`: This field represents the molecule's name according to the International Union of Pure and Applied Chemistry's systematic naming conventions. For example, \"3,4-dihydroxybenzoic acid geranyl ester\" clearly describes the chemical structure of the compound in a standardized way. `bioActivity`: Here, you detail the molecule's biological effect or function. \"Inhibition of Protease\" means the compound prevents or reduces the activity of protease enzymes, crucial for understanding its potential therapeutic uses. `collectionSpecies`: This specifies the biological source or species from which the molecule was isolated, such as \"Piper crassinervium (Piperaceae)\", pointing to a specific plant within the Piperaceae family. `collectionSite`: Indicates the geographical origin where the compound was collected or the organism was found. \"Araraquara/SP\" refers to a location in São Paulo, Brazil, providing context for the environmental conditions of the source. `collectionType`: Describes the origin or process through which the compound was obtained, such as \"Biotransformation Product\", indicating the compound results from a biological organism chemically modifying a precursor compound. Your analysis should be precise, adhering to the JSON format provided, with each field filled according to the information available from the paper. If certain details are not mentioned, leave the fields empty with `""`. If there are multiple molecules, provide information for each one separately. The keys are the same for each molecule, but the values will differ based on the information available in the document. Ensure that the JSON format is maintained for each molecule. Name the molecules as Molecule_1, Molecule_2, and so on. Please find as much information as possible about each molecule in the document."
    )

    client = OpenAI()
    client.api_key = os.environ["OPENAI_API_KEY"]
    response_list = ""
    response_res = ""

    for i in range(0, k):
        user_message = (
            f"Based on the document content: {pages}, and previous analyses: {response_list} but check if its right otherwise change it, identify the described molecule enrich it with information from the above. If specific information is not available, leave the field empty."
        )
        response = client.chat.completions.create(
            messages=[
                {"role": "system", "content": system_message},
                {"role": "user", "content": user_message}
            ],
            seed=920987036854775807,
            stream=True,
            response_format={"type": "json_object"},
            model="gpt-4-1106-preview",
        )

        response_string = ""
        for chunk in response:
            json_array_one.append(chunk.choices[0].delta.content or "")
            response_res += chunk.choices[0].delta.content or ""
            response_string += chunk.choices[0].delta.content or ""
            #print(chunk.choices[0].delta.content or "", end="")
            response_list = response_list + str(chunk.choices[0].delta.content) or ""

    return response_res


import pandas as pd

pdf = "pdfs/10.1016@0031-9422(73)85034-4.pdf"
df2 = pd.DataFrame(columns=["pdf", "output"])

stream = call_openai_api(pdf)
stream = stream.replace("\n", "")
df2 = df2._append({"pdf": pdf, "output": stream}, ignore_index=True)

df2.to_csv(pdf + ".csv")

In [ ]:
import time
import os
import pandas as pd


def process_pdfs_in_folder(folder_path):
    # Initialize an empty DataFrame for consolidated data
    consolidated_df = pd.DataFrame(columns=["pdf", "output"])
    i =0
    # List all PDF files in the specified folder
    pdf_files = [f for f in os.listdir(folder_path) if f.endswith('.pdf')]

    # Process each PDF file
    for pdf_file in pdf_files:
        i += 1
        print(f"Processing PDF {i}/{len(pdf_files)}: {pdf_file}")
        pdf_path = os.path.join(folder_path, pdf_file)
        try:
            stream = call_openai_api(pdf_path)
            stream = stream.replace("\n", "")
            # Create a DataFrame for the current PDF
            df_current = pd.DataFrame([{"pdf": pdf_path, "output": stream}])
            # Append the current DataFrame to the consolidated DataFrame
            consolidated_df = pd.concat([consolidated_df, df_current], ignore_index=True)
            # Export the current DataFrame to an individual CSV file
            csv_filename = os.path.join(folder_path, pdf_file + ".csv")
            df_current.to_csv(csv_filename, index=False)
        except Exception as e:
            print(f"Error processing {pdf_path}: {e}")
            # sleep for 60 seconds to avoid hitting the OpenAI API rate limit
            time.sleep(60)

    # Export the consolidated DataFrame to a CSV file
    consolidated_csv_filename = os.path.join(folder_path, "consolidated_output.csv")
    consolidated_df.to_csv(consolidated_csv_filename, index=False)


# Process PDFs in the specified folder
process_pdfs_in_folder('pdfs/')


In [ ]:
consolidated_df = pd.read_csv('consolidated_output_full.csv')
#print(consolidated_df)
# iterate over the row pdf of the DataFrame and search if the pdfs of the folder are in the row pdf
# if they are not, then print the pdf name and add them to the list of pdfs not processed
not_processed_pdfs = []
for pdf in os.listdir('pdfs/'):
    if pdf.endswith('.pdf'):
        if "pdfs/"+pdf not in consolidated_df['pdf'].values:
            print(f"PDF {pdf} not processed.")
            not_processed_pdfs.append(pdf)


In [ ]:
# call again for the pdfs not processed and add them to the consolidated_df
for pdf in not_processed_pdfs:
    pdf_path = os.path.join('pdfs/', pdf)
    try:
        stream = call_openai_api(pdf_path)
        stream = stream.replace("\n", "")
        df_current = pd.DataFrame([{"pdf": pdf_path, "output": stream}])
        consolidated_df = pd.concat([consolidated_df, df_current], ignore_index=True)
        csv_filename = os.path.join('pdfs/', pdf + ".csv")
        df_current.to_csv(csv_filename, index=False)
    except Exception as e:
        print(f"Error processing {pdf_path}: {e}")
        time.sleep(60)

In [ ]:
# save the consolidated_df to a csv file
consolidated_df.to_csv('consolidated_output_full.csv', index=False)

Processing PDF 1/392: 10.1590@S0103-50532001000300014.pdf
Processing PDF 2/392: 10.1021@jf60217a066.pdf
Processing PDF 3/392: 10.1590@S0100-40422012001100015.pdf
Processing PDF 4/392: 10.1016@S0031-9422(01)00079-6.pdf
Processing PDF 5/392: 10.3390@molecules15096140.pdf
Processing PDF 6/392: 10.1016@0031-9422(95)00969-8.pdf
Processing PDF 7/392: 10.1016@S0031-9422(00)98664-3.pdf
Processing PDF 8/392: 10.1351@pac200173030617.pdf
Processing PDF 9/392: 10.1002@cbdv.200590016.pdf
Processing PDF 10/392: 10.1016@0031-9422(73)80047-0.pdf
Processing PDF 11/392: 10.1515@znc-2011-5-607.pdf
Processing PDF 12/392: 10.1007@s10295-012-1165-2.pdf
Processing PDF 13/392: 10.1016@S0031-9422(00)89350-4.pdf
Processing PDF 14/392: 10.1016@S0031-9422(00)88971-2.pdf
Processing PDF 15/392: 10.1016@0031-9422(81)85112-6.pdf
Processing PDF 16/392: 10.1021@np400717m.pdf
Processing PDF 17/392: 10.1590@S0103-50532005000800023.pdf
Processing PDF 18/392: 10.1016@j.cbi.2010.08.008.pdf
Processing PDF 19/392: 10.1590@S0100-40422002000700009.pdf
Processing PDF 20/392: 10.1021@np9704617.pdf
Processing PDF 21/392: 10.1016@0031-9422(81)80027-1.pdf
Processing PDF 22/392: 10.1016@S0031-9422(03)00153-5.pdf
Processing PDF 23/392: 10.1039@c3cc46213g.pdf
Processing PDF 24/392: 10.1016@S0031-9422(03)00109-2.pdf
Processing PDF 25/392: 10.1016@0031-9422(73)85045-9.pdf
Processing PDF 26/392: 10.1007@s00044-008-9161-9.pdf
Processing PDF 27/392: 10.1590@S0100-40422009000100025.pdf
Processing PDF 28/392: 10.1016@0305-1978(92)90104-L.pdf
Processing PDF 29/392: 10.1590@S0100-40422002000300002.pdf
Processing PDF 30/392: 10.1515@znc-2008-5-611.pdf
Processing PDF 31/392: 10.1016@0305-1978(88)90060-9.pdf
Processing PDF 32/392: 10.1016@0031-9422(95)00504-Z.pdf
Processing PDF 33/392: 10.1590@S0100-40422004000200005.pdf
Processing PDF 34/392: 10.1590@S0100-40422010000200017.pdf
Processing PDF 35/392: 10.1016@0031-9422(81)85097-2.pdf
Processing PDF 36/392: 10.1023@A_1022634012212.pdf
Processing PDF 37/392: 10.1021@np900644x.pdf
Processing PDF 38/392: 10.1016@0031-9422(74)80351-1.pdf
Processing PDF 39/392: 10.1016@0040-4020(72)80138-8.pdf
Processing PDF 40/392: 10.1016@0031-9422(75)85051-5.pdf
Processing PDF 41/392: 10.1515@znc-2002-3-433.pdf
Processing PDF 42/392: 10.1021@jo00200a028.pdf
Processing PDF 43/392: 10.1590@S0103-50532006000300017.pdf
Processing PDF 44/392: 10.1016@S0031-9422(99)00023-0.pdf
Processing PDF 45/392: 10.1107@S160053680302107X.pdf
Processing PDF 46/392: 10.1078@0944-7113-00016.pdf
Processing PDF 47/392: 10.1016@S0031-9422(00)81981-0.pdf
Processing PDF 48/392: 10.1016@0031-9422(75)83050-0.pdf
Processing PDF 49/392: 10.1016@S0031-9422(00)97301-1.pdf
Processing PDF 50/392: 10.1590@S0100-40422009000500005.pdf
Error processing pdfs/10.1590@S0100-40422009000500005.pdf: [Errno 54] Connection reset by peer
Processing PDF 51/392: 10.1590@S0103-50532005000800008.pdf
Processing PDF 52/392: 10.1016@0031-9422(91)85077-D.pdf
Processing PDF 53/392: 10.1016@0031-9422(93)80055-W.pdf
Processing PDF 54/392: 10.1021@np50024a022.pdf
Processing PDF 55/392: 10.1590@S0100-40422010001000024.pdf
Processing PDF 56/392: 10.1016@S0031-9422(99)00532-4.pdf
Processing PDF 57/392: 10.1590@S0073-47212009000100009.pdf
Processing PDF 58/392: 10.1016@j.phytochem.2008.01.006.pdf
Processing PDF 59/392: 10.1002@elps.201000658.pdf
Processing PDF 60/392: 10.1021@np50006a014.pdf
Processing PDF 61/392: 10.1016@j.phytochem.2005.05.019.pdf
Processing PDF 62/392: 10.1016@0031-9422(82)83121-X.pdf
Processing PDF 63/392: 10.1590@S0103-50531999000400011.pdf
Processing PDF 64/392: 10.1016@S0031-9422(99)00105-3.pdf
Processing PDF 65/392: 10.1016@S0031-9422(00)00323-X.pdf
Processing PDF 66/392: 10.1016@0031-9422(80)83082-2.pdf
Processing PDF 67/392: 10.1016@S0031-9422(00)97481-8.pdf
Processing PDF 68/392: 10.1002@pca.902.pdf
Processing PDF 69/392: 10.1590@S0102-695X2010005000017.pdf
Processing PDF 70/392: 10.1590@S0103-50532009000100029.pdf
Processing PDF 71/392: 10.1021@jf00125a014.pdf
Processing PDF 72/392: 10.1080@14786410701582282.pdf
Processing PDF 73/392: 10.1139@v81-400.pdf
Processing PDF 74/392: 10.1021@jf00120a031.pdf
Processing PDF 75/392: 10.1016@S0031-9422(00)89656-9.pdf
Processing PDF 76/392: 10.1590@S0103-50532005000800025.pdf
Processing PDF 77/392: 10.1021@jf100233y.pdf
Processing PDF 78/392: 10.1021@np049863m.pdf
Processing PDF 79/392: 10.1016@j.bse.2012.03.005.pdf
Processing PDF 80/392: 10.1016@j.phytol.2013.08.014.pdf
Processing PDF 81/392: 10.1590@S0103-50532005000800018.pdf
Processing PDF 82/392: 10.1590@S0102-695X2008000200010.pdf
Processing PDF 83/392: 10.1590@S0103-50532004000300008.pdf
Processing PDF 84/392: 10.1002@cbdv.201400065.pdf
Processing PDF 85/392: 10.1016@S0031-9422(00)94440-6.pdf
Processing PDF 86/392: 10.1016@S0040-4020(01)88671-3.pdf
Processing PDF 87/392: 10.1016@j.jphotobiol.2013.01.006.pdf
Processing PDF 88/392: 10.1002@hlca.200890233.pdf
Processing PDF 89/392: 10.1590@S0100-40422012001100006.pdf
Processing PDF 90/392: 10.1016@0031-9422(74)85041-7.pdf
Processing PDF 91/392: 10.1590@S0100-40422012001100004.pdf
Processing PDF 92/392: 10.1016@S0031-94220000323-X.pdf
Processing PDF 93/392: 10.1055@s-0034-1368301.pdf
Processing PDF 94/392: 10.1590@S0103-50532009000400008.pdf
Processing PDF 95/392: 10.1016@S0031-9422(98)00069-7.pdf
Processing PDF 96/392: 10.1016@S0031-9422(00)97240-6.pdf
Processing PDF 97/392: 10.1016@0031-9422(74)80106-8.pdf
Processing PDF 98/392: 10.1016@0031-9422(95)00368-H.pdf
Processing PDF 99/392: 10.1002@hlca.200890147.pdf
Processing PDF 100/392: 10.5935@0103-5053.19930018.pdf
Processing PDF 101/392: 10.1016@S0031-9422(00)00225-9.pdf
Processing PDF 102/392: 10.1590@S0103-50532005000800026.pdf
Processing PDF 103/392: 10.1016@j.bmc.2007.11.008.pdf
Processing PDF 104/392: 10.1590@S0100-40422010000900005.pdf
Processing PDF 105/392: 10.1016@S0031-9422(00)83834-0.pdf
Processing PDF 106/392: 10.1590@S0103-50532006000200010.pdf
Processing PDF 107/392: 10.1590@S0100-40422008000400005.pdf
Processing PDF 108/392: 10.1590@S0103-50532005000300018.pdf
Processing PDF 109/392: 10.5935@0100-4042.20140049.pdf
Processing PDF 110/392: 10.3390@molecules15129205.pdf
Processing PDF 111/392: 10.1107@S1600536803019913.pdf
Processing PDF 112/392: 10.1590@S0103-50532001000300010.pdf
Processing PDF 113/392: 10.1016@S0031-9422(00)91125-7.pdf
Processing PDF 114/392: 10.1080@14786419.2011.647020.pdf
Processing PDF 115/392: 10.1016@0031-9422(80)83022-6.pdf
Processing PDF 116/392: 10.1016@S0031-9422(00)89849-0.pdf

Advanced encoding [] not implemented yet
Advanced encoding [] not implemented yet
Advanced encoding [] not implemented yet
Advanced encoding [] not implemented yet
Advanced encoding [] not implemented yet
Advanced encoding [] not implemented yet
Advanced encoding [] not implemented yet
Advanced encoding [] not implemented yet

Processing PDF 117/392: 10.1055@s-0029-1185364.pdf
Processing PDF 118/392: 10.1590@S0103-50532002000100010.pdf
Processing PDF 119/392: 10.1515@znb-2005-0715.pdf
Processing PDF 120/392: 10.1016@S0031-9422(97)00598-0.pdf
Processing PDF 121/392: 10.1016@S0031-9422(00)80173-9.pdf
Processing PDF 122/392: 10.1016@j.jfca.2003.09.013.pdf
Processing PDF 123/392: 10.1590@S0100-40422009000600037.pdf
Processing PDF 124/392: 10.1016@0031-9422(83)80035-1.pdf
Processing PDF 125/392: 10.1016@j.phytochem.2011.12.016.pdf
Processing PDF 126/392: 10.1016@j.phytochem.2005.06.019.pdf
Processing PDF 127/392: 10.1590@1809-43921982123615.pdf
Processing PDF 128/392: 10.1016@S0031-9422(00)89351-6.pdf
Processing PDF 129/392: 10.1016@S0031-9422(00)83467-6.pdf
Processing PDF 130/392: 10.1016@S0031-9422(00)86522-X.pdf
Processing PDF 131/392: 10.1590@S0100-40422003000500003.pdf
Processing PDF 132/392: 10.1016@0031-9422(83)83046-5.pdf
Processing PDF 133/392: 10.1016@S0031-9422(98)00249-0.pdf
Processing PDF 134/392: 10.1021@np0303963.pdf
Processing PDF 135/392: 10.1016@j.phytochem.2010.05.012.pdf
Processing PDF 136/392: 10.1021@np020547m.pdf
Processing PDF 137/392: 10.1016@S0031-9422(00)84396-4.pdf
Processing PDF 138/392: 10.1248@bpb.31.538.pdf
Processing PDF 139/392: 10.1016@S0031-9422(00)82898-8.pdf
Processing PDF 140/392: 10.1016@0031-9422(81)84045-9.pdf
Processing PDF 141/392: 10.1002@ps.478.pdf
Processing PDF 142/392: 10.1590@S0103-50532005000400031.pdf
Processing PDF 143/392: 10.1590@S0100-40422007000500009.pdf
Processing PDF 144/392: 10.1021@np500453x.pdf
Processing PDF 145/392: 10.3987@COM-09-11671.pdf
Processing PDF 146/392: 10.1016@0031-9422(76)85091-1.pdf
Processing PDF 147/392: 10.1080@14786410802243271.pdf
Processing PDF 148/392: 10.1016@j.phytochem.2007.08.012.pdf
Processing PDF 149/392: 10.1016@j.jep.2007.09.020.pdf
Processing PDF 150/392: 10.1016@0031-9422(81)85198-9.pdf
Processing PDF 151/392: 10.1021@np9703656.pdf
Processing PDF 152/392: 10.1039@c39720000514.pdf
Processing PDF 153/392: 10.1016@0031-9422(76)90000-5.pdf
Processing PDF 154/392: 10.1016@S0040-4039(00)85815-3.pdf
Processing PDF 155/392: 10.1016@j.tiv.2008.01.003.pdf
Processing PDF 156/392: 10.1016@0031-9422(95)00464-I.pdf
Processing PDF 157/392: 10.1016@0031-9422(75)85303-9.pdf
Processing PDF 158/392: 10.1515@znc-2002-5-602.pdf
Processing PDF 159/392: 10.1016@j.phytochem.2005.04.043.pdf
Processing PDF 160/392: 10.1590@S0100-40421998000600005.pdf
Processing PDF 161/392: 10.1021@jf00030a035.pdf
Processing PDF 162/392: 10.1016@0031-9422(80)87033-6.pdf
Processing PDF 163/392: 10.1016@0031-9422(73)85048-4.pdf
Processing PDF 164/392: 10.2174@187152010794728620.pdf
Processing PDF 165/392: 10.1590@S0103-50531999000300013.pdf
Processing PDF 166/392: 10.1093@mutage@gep034.pdf
Processing PDF 167/392: 10.1016@0031-9422(81)85172-2.pdf
Processing PDF 168/392: 10.1016@0031-9422(73)85034-4.pdf
Processing PDF 169/392: 10.1590@S0103-50532009000200025.pdf
Processing PDF 170/392: 10.1016@j.phytochem.2007.04.025.pdf
Processing PDF 171/392: 10.1016@j.phytochem.2006.06.004.pdf
Processing PDF 172/392: 10.1016@j.bse.2008.05.001.pdf
Processing PDF 173/392: 10.1016@j.chroma.2009.02.066.pdf
Processing PDF 174/392: 10.1590@S0100-46702007000300002.pdf
Processing PDF 175/392: 10.1002@ps.1278.pdf
Processing PDF 176/392: 10.1016@S0031-9422(97)00824-8.pdf
Processing PDF 177/392: 10.1590@1809-43921982124801.pdf
Processing PDF 178/392: 10.1016@j.bse.2003.09.006.pdf
Processing PDF 179/392: 10.1016@S0031-9422(00)86797-7.pdf
Processing PDF 180/392: 10.1016@j.phytol.2012.05.004.pdf
Processing PDF 181/392: 10.1021@np4006822.pdf
Processing PDF 182/392: 10.1590@S0100-40422009000800010.pdf
Processing PDF 183/392: 10.1590@1809-43921984143453.pdf
Processing PDF 184/392: 10.1590@S0103-50532006000500017.pdf
Processing PDF 185/392: 10.1016@j.jpba.2007.05.018.pdf
Processing PDF 186/392: 10.1016@0031-9422(91)83479-5.pdf
Processing PDF 187/392: 10.1021@np1007476.pdf
Processing PDF 188/392: 10.1016@j.micres.2004.06.003.pdf
Processing PDF 189/392: 10.1016@0031-9422(92)80374-N.pdf
Processing PDF 190/392: 10.1590@1809-43921983132409.pdf
Processing PDF 191/392: 10.1016@0031-9422(80)83081-0.pdf
Processing PDF 192/392: 10.1016@0031-9422(81)84067-8.pdf
Processing PDF 193/392: 10.1016@S0031-9422(00)80162-4.pdf
Processing PDF 194/392: 10.1016@S0031-9422(96)00571-7.pdf
Processing PDF 195/392: 10.1016@S0031-9422(00)91124-5.pdf
Processing PDF 196/392: 10.1021@np800612x.pdf
Processing PDF 197/392: 10.1021@np0496845.pdf
Processing PDF 198/392: 10.1016@S0031-9422(96)00747-9.pdf
Processing PDF 199/392: 10.1002@bmc.1155.pdf
Processing PDF 200/392: 10.1248@cpb.56.723.pdf
Processing PDF 201/392: 10.1365@s10337-009-1338-y.pdf
Processing PDF 202/392: 10.1016@S0031-9422(82)85050-4.pdf
Processing PDF 203/392: 10.1021@np800106x.pdf
Processing PDF 204/392: 10.1021@np030530j.pdf
Processing PDF 205/392: 10.1021@jf801103p.pdf
Processing PDF 206/392: 10.1016@S0031-9422(00)00477-5.pdf
Processing PDF 207/392: 10.1016@S0031-9422(01)00431-9.pdf
Processing PDF 208/392: 10.1016@S0031-9422(00)80444-6.pdf
Processing PDF 209/392: 10.1016@j.carres.2008.07.014.pdf
Processing PDF 210/392: 10.1016@0031-9422(73)85046-0.pdf
Processing PDF 211/392: 10.1016@S0031-9422(97)00238-0.pdf
Processing PDF 212/392: 10.1016@S0031-9422(00)88981-5.pdf
Processing PDF 213/392: 10.1590@S0103-50532002000100019.pdf
Processing PDF 214/392: 10.1021@np50036a007.pdf
Processing PDF 215/392: 10.1590@S0100-40422008000600038.pdf
Processing PDF 216/392: 10.1007@s00253-007-1171-x.pdf
Processing PDF 217/392: 10.1590@S0102-695X2006000200006.pdf
Processing PDF 218/392: 10.1590@S1516-89132011000100008.pdf
Processing PDF 219/392: 10.3390@molecules190812031.pdf
Processing PDF 220/392: 10.1002@chir.20676.pdf
Processing PDF 221/392: 10.1055@s-2002-36346.pdf
Processing PDF 222/392: 10.1016@S0031-9422(00)00298-3.pdf
Processing PDF 223/392: 10.1016@S0031-9422(97)00167-2.pdf
Processing PDF 224/392: 10.1016@S0031-9422(00)94260-2.pdf
Processing PDF 225/392: 10.1515@znc-2009-11-1212.pdf
Processing PDF 226/392: 10.1016@0031-9422(75)85047-3.pdf
Processing PDF 227/392: 10.1107@s0108270183006149.pdf
Processing PDF 228/392: 10.1016@S0031-9422(03)00006-2.pdf
Processing PDF 229/392: 10.1590@S0102-695X2009000100002.pdf
Processing PDF 230/392: 10.1016@0031-9422(75)85143-0.pdf
Processing PDF 231/392: 10.1016@S0040-4039(00)94036-X.pdf
Processing PDF 232/392: 10.1016@S0031-9422(00)88985-2.pdf
Processing PDF 233/392: 10.1016@S0031-9422(00)89244-4.pdf
Processing PDF 234/392: 10.1016@0031-9422(74)80105-6.pdf
Processing PDF 235/392: 10.1590@S0102-695X2007000300003.pdf
Processing PDF 236/392: 10.1093@chromsci@bms163.pdf
Processing PDF 237/392: 10.1016@S0031-9422(00)00285-5.pdf
Processing PDF 238/392: 10.1021@np030100t.pdf
Processing PDF 239/392: 10.1021@np50086a012.pdf
Processing PDF 240/392: 10.1002@pca.818.pdf
Processing PDF 241/392: 10.1590@S0103-50532005000400005.pdf
Processing PDF 242/392: 10.1021@cc100076k.pdf
Processing PDF 243/392: 10.1016@0031-9422(80)83207-9.pdf
Processing PDF 244/392: 10.1016@j.tetasy.2010.09.004.pdf
Processing PDF 245/392: 10.1016@0031-94229500504-Z.pdf
Processing PDF 246/392: 10.1016@0031-9422(82)83181-6.pdf
Processing PDF 247/392: 10.1016@S0031-9422(00)94671-5.pdf
Processing PDF 248/392: 10.1590@S0102-695X2012005000106.pdf
Processing PDF 249/392: 10.1016@j.phytochem.2009.08.013.pdf
Processing PDF 250/392: 10.1016@0031-9422(92)83701-Y.pdf
Processing PDF 251/392: 10.1590@S0103-50532003000500012.pdf
Processing PDF 252/392: 10.1021@np50032a021.pdf
Processing PDF 253/392: 10.1016@0031-9422(74)80252-9.pdf
Processing PDF 254/392: 10.1016@S0031-9422(00)89841-6.pdf
Processing PDF 255/392: 10.1016@S0031-9422(00)85606-X.pdf
Processing PDF 256/392: 10.1016@j.bse.2005.10.012.pdf
Processing PDF 257/392: 10.1016@S0031-9422(00)84995-X.pdf
Processing PDF 258/392: 10.1002@cbdv.201000011.pdf
Processing PDF 259/392: 10.1016@S0031-9422(00)91467-5.pdf
Processing PDF 260/392: 10.1016@j.tetlet.2008.06.124.pdf
Processing PDF 261/392: 10.1016@S0031-9422(00)91063-X.pdf
Processing PDF 262/392: 10.1016@0031-9422(73)80421-2.pdf
Processing PDF 263/392: 10.1016@S0031-9422(00)95115-X.pdf
Processing PDF 264/392: 10.1016@0031-9422(92)83738-K.pdf
Processing PDF 265/392: 10.1016@S0031-9422(00)88490-3.pdf
Processing PDF 266/392: 10.1016@S0031-9422(00)95305-6.pdf
Processing PDF 267/392: 10.1002@cbdv.200800342.pdf
Processing PDF 268/392: 10.1023@B_JOEC.0000042400.14451.08.pdf
Processing PDF 269/392: 10.1590@s0100-40422008000700015.pdf
Processing PDF 270/392: 10.1016@S0031-9422(00)81975-5.pdf
Processing PDF 271/392: 10.1590@S0100-40422005000400009.pdf
Processing PDF 272/392: 10.1002@elps.200900216.pdf
Processing PDF 273/392: 10.1016@j.tetasy.2009.11.001.pdf
Processing PDF 274/392: 10.1016@S0031-9422(00)94227-4.pdf
Processing PDF 275/392: 10.1515@znb-2005-0321.pdf
Processing PDF 276/392: 10.1021@np100840w.pdf
Processing PDF 277/392: 10.1590@S0103-50532004000400004.pdf
Processing PDF 278/392: 10.1590@S0103-50532010000500016.pdf
Processing PDF 279/392: 10.1016@S0031-9422(97)00872-8.pdf
Processing PDF 280/392: 10.1016@j.bmc.2005.04.030.pdf
Processing PDF 281/392: 10.3998@ark.5550190.0005.606.pdf
Processing PDF 282/392: 10.1016@S0031-9422(00)84055-8.pdf
Processing PDF 283/392: 10.1021@np050522y.pdf
Error processing pdfs/10.1021@np050522y.pdf: [Errno 54] Connection reset by peer
Processing PDF 284/392: 10.1016@0031-9422(95)00093-3.pdf
Processing PDF 285/392: 10.4067@S0717-97072009000300004.pdf
Processing PDF 286/392: 10.1016@0040-4020(95)00254-6.pdf
Processing PDF 287/392: 10.1016@j.phytochem.2013.09.022.pdf
Processing PDF 288/392: 10.1016@S0031-9422(00)00226-0.pdf
Processing PDF 289/392: 10.1002@pca.1208.pdf
Processing PDF 290/392: 10.1016@j.phytol.2010.11.003.pdf
Processing PDF 291/392: 10.1002@jobm.200800093.pdf
Processing PDF 292/392: 10.1590@S0100-40422004000600007.pdf
Processing PDF 293/392: 10.1016@S0031-9422(00)85714-3.pdf
Processing PDF 294/392: 10.1039@c0ay00008f.pdf
Processing PDF 295/392: 10.1016@S0040-4039(01)97216-8.pdf
Processing PDF 296/392: 10.1248@cpb.56.407.pdf
Processing PDF 297/392: 10.1016@0031-9422(81)84071-X.pdf
Processing PDF 298/392: 10.1016@0031-9422(81)85125-4.pdf
Processing PDF 299/392: 10.1016@S0031-9422(00)00248-X.pdf
Processing PDF 300/392: 10.1590@S0100-40422006000300016.pdf
Processing PDF 301/392: 10.1590@S0103-50532005000800016.pdf
Processing PDF 302/392: 10.1016@0031-9422(92)90048-U.pdf
Processing PDF 303/392: 10.1021@np200110f.pdf
Processing PDF 304/392: 10.1016@S0031-9422(00)89654-5.pdf
Processing PDF 305/392: 10.1016@0305-0491(82)90526-0.pdf
Processing PDF 306/392: 10.1590@S0103-50532003000300007.pdf
Processing PDF 307/392: 10.1590@S0100-40422010001000006.pdf
Processing PDF 308/392: 10.1023@A_1025694720727.pdf
Processing PDF 309/392: 10.1590@S0102-695X2010000100002.pdf
Processing PDF 310/392: 10.1016@0305-1978(78)90024-8.pdf
Processing PDF 311/392: 10.1002@chin.200552195.pdf
Processing PDF 312/392: 10.5935@0103-5053.19960024.pdf
Processing PDF 313/392: 10.1590@S0103-50532002000300014.pdf
Processing PDF 314/392: 10.1590@S0100-40422000000300004.pdf
Processing PDF 315/392: 10.1016@S0031-9422(99)00132-6.pdf
Processing PDF 316/392: 10.1021@np200556p.pdf
Processing PDF 317/392: 10.1021@np0103970.pdf
Processing PDF 318/392: 10.1590@S0103-50532001000200022.pdf
Processing PDF 319/392: 10.1016@j.phytochem.2006.09.005.pdf
Processing PDF 320/392: 10.1016@j.jpba.2012.08.028.pdf
Processing PDF 321/392: 10.1016@0031-9422(83)80034-X.pdf
Processing PDF 322/392: 10.1021@np50017a019.pdf
Processing PDF 323/392: 10.1016@S0031-9422(00)98552-2.pdf
Processing PDF 324/392: 10.1016@S0031-9422(99)00197-1.pdf
Processing PDF 325/392: 10.1590@S0100-40422008000400023.pdf
Processing PDF 326/392: 10.1016@S0031-9422(00)84183-7.pdf
Processing PDF 327/392: 10.1016@j.jchromb.2012.07.013.pdf
Processing PDF 328/392: 10.1590@S1519-566X2008000600013.pdf
Processing PDF 329/392: 10.1021@np060254j.pdf
Processing PDF 330/392: 10.1590@S0103-50532005000800029.pdf
Processing PDF 331/392: 10.1016@j.chroma.2009.02.009.pdf
Processing PDF 332/392: 10.1080@14786410412331271979.pdf
Processing PDF 333/392: 10.1055@s-0028-1088323.pdf
Processing PDF 334/392: 10.1590@S0103-50532000000600012.pdf
Processing PDF 335/392: 10.1016@S0031-9422(00)90130-4.pdf
Processing PDF 336/392: 10.1021@np070312g.pdf
Processing PDF 337/392: 10.1155@2013@835081.pdf
Processing PDF 338/392: 10.1016@S0031-9422(00)89891-X.pdf
Processing PDF 339/392: 10.1590@1809-43921982123668.pdf
Processing PDF 340/392: 10.1016@j.phytochem.2004.03.039.pdf
Processing PDF 341/392: 10.1590@S0103-50531999000100012.pdf
Processing PDF 342/392: 10.1021@np50017a018.pdf
Processing PDF 343/392: 10.1016@0031-9422(94)00568-E.pdf
Processing PDF 344/392: 10.1016@S0031-9422(00)89245-6.pdf
Processing PDF 345/392: 10.1016@S0031-9422(00)82735-1.pdf
Processing PDF 346/392: 10.1002@ptr.1249.pdf
Processing PDF 347/392: 10.1016@S0031-9422(00)89914-8.pdf
Processing PDF 348/392: 10.1002@hlca.200790187.pdf
Processing PDF 349/392: 10.1080@14786410903335232.pdf
Processing PDF 350/392: 10.1016@0031-9422(90)80024-B.pdf
Processing PDF 351/392: 10.1002@pca.843.pdf
Processing PDF 352/392: 10.1016@S0031-9422(00)81526-5.pdf
Processing PDF 353/392: 10.1016@0031-9422(92)80271-F.pdf
Processing PDF 354/392: 10.1016@0305-1978(94)90107-4.pdf
Processing PDF 355/392: 10.1016@0031-9422(73)80714-9.pdf
Processing PDF 356/392: 10.1016@0031-9422(75)80372-4.pdf
Processing PDF 357/392: 10.1590@S0100-40422012000700020.pdf
Processing PDF 358/392: 10.1590@S0100-40422010001000014.pdf
Processing PDF 359/392: 10.1016@0031-9422(75)85390-8.pdf
Processing PDF 360/392: 10.1016@0031-9422(75)85358-1.pdf
Processing PDF 361/392: 10.1016@0031-9422(79)80188-0.pdf
Processing PDF 362/392: 10.1016@j.bmc.2010.12.056.pdf
Processing PDF 363/392: 10.1016@0031-9422(80)83208-0.pdf
Processing PDF 364/392: 10.1590@S0103-50532007000700017.pdf
Processing PDF 365/392: 10.1021@jf063451x.pdf
Processing PDF 366/392: 10.1016@j.bse.2006.06.007.pdf
Processing PDF 367/392: 10.1016@S0031-9422(00)89084-6.pdf
Processing PDF 368/392: 10.1016@S0031-9422(97)00140-4.pdf
Processing PDF 369/392: 10.1080@14786410600907002.pdf
Processing PDF 370/392: 10.1016@S0031-9422(00)89347-4.pdf
Processing PDF 371/392: 10.1016@S0014-5793(02)02700-X.pdf
Processing PDF 372/392: 10.1016@S0031-9422(00)85161-4.pdf
Processing PDF 373/392: 10.1016@S0031-9422(00)80552-X.pdf
Processing PDF 374/392: 10.1016@S0031-9422(01)00315-6.pdf
Processing PDF 375/392: 10.1016@S0031-9422(00)84061-3.pdf
Processing PDF 376/392: 10.1021@np9004079.pdf
Processing PDF 377/392: 10.1016@0031-9422(74)80100-7.pdf
Processing PDF 378/392: 10.1016@S0031-9422(00)84426-X.pdf
Processing PDF 379/392: 10.1590@S0100-40422005000600011.pdf
Processing PDF 380/392: 10.1016@j.biortech.2010.01.041.pdf
Processing PDF 381/392: 10.1590@S0103-50532005000800007.pdf
Processing PDF 382/392: 10.1002@bscb.19810900913.pdf
Processing PDF 383/392: 10.1016@S0031-9422(00)88692-6.pdf
Processing PDF 384/392: 10.1590@S0103-50531998000100007.pdf
Processing PDF 385/392: 10.1016@0031-9422(75)85324-6.pdf
Processing PDF 386/392: 10.1590@S0100-40422005000600010.pdf
Processing PDF 387/392: 10.1016@0031-9422(91)85060-D.pdf
Processing PDF 388/392: 10.1590@S0100-40422008000400018.pdf
Processing PDF 389/392: 10.1590@S0103-50532011000700021.pdf
Processing PDF 390/392: 10.1590@S0100-46702006000400008.pdf
Processing PDF 391/392: 10.1016@S0031-9422(00)88765-8.pdf
Processing PDF 392/392: 10.1016@0031-9422(81)80042-8.pdf


In [ ]:
cosol
